Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic
для источников paid и email из России - ставим ad
для источников paid и email не из России - ставим other
все остальные варианты берем из traffic_source без изменений

In [2]:
import pandas as pd
data = pd.read_csv('visit_log.csv', sep=';')

In [3]:
data.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [4]:
def typesource(region, traffic_source):
    if traffic_source == 'yandex' or traffic_source == 'google':
        return 'organic'
    elif traffic_source == 'paid' or traffic_source == 'email':
        if region == 'Russia':
            return 'ad'
        else:
            return 'other'
    else:
        return traffic_source
    

In [5]:
data['source_type'] = data.apply(lambda row: typesource(row['region'], row['traffic_source']), axis = 1)

In [7]:
data.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [16]:
urls = pd.read_table('URLs.txt')

In [17]:
urls.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [26]:
urls.loc[urls['url'].str.contains(r'/\d{8}-', case = False)]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
rating = pd.read_csv('ratings.csv')
rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
rating_count = rating.groupby('userId')[['rating']].count()
big_rating = rating_count[rating_count.rating>100]
big_rating

,rating
userId,
4,204
8,116
15,1700
17,363
19,423
...,...
656,128
659,142
664,519


In [34]:
maxmintime = rating.pivot_table(index = 'userId', values = 'timestamp', aggfunc = ['max', 'min'], fill_value = 0)
maxmintime.columns = ["_".join((j,k)) for j,k in maxmintime.columns]
maxmintime.reset_index()

,userId,max_timestamp,min_timestamp
0,1,1260759205,1260759108
1,2,835356246,835355395
2,3,1298932787,1298861589
3,4,949982274,949778714
4,5,1163375145,1163373044
...,...,...,...
666,667,847272234,847271220
667,668,993613478,993613196
668,669,1015829766,1015829081
669,670,940944033,938781328


In [35]:
joinrating = big_rating.join(maxmintime, how = 'left')
joinrating

,rating,max_timestamp,min_timestamp
4,204,949982274,949778714
8,116,1154474527,1154389340
15,1700,1469330735,997937239
17,363,1127476640,1127468587
19,423,855195373,855190091
...,...,...,...
656,128,986244044,986240991
659,142,866207451,834598040
664,519,1441911722,1343731283
665,434,1046967769,992836298


In [39]:
joinrating['Diff'] = joinrating['max_timestamp']-joinrating['min_timestamp']
joinrating

,rating,max_timestamp,min_timestamp,Diff
4,204,949982274,949778714,203560
8,116,1154474527,1154389340,85187
15,1700,1469330735,997937239,471393496
17,363,1127476640,1127468587,8053
19,423,855195373,855190091,5282
...,...,...,...,...
656,128,986244044,986240991,3053
659,142,866207451,834598040,31609411
664,519,1441911722,1343731283,98180439
665,434,1046967769,992836298,54131471


Задание 4
Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов
Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента

In [3]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [4]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [6]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [7]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [12]:
revenue = rzd.merge(auto, how = 'outer', on = 'client_id').merge(air, how = 'outer', on = 'client_id')
revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [13]:
revenueaddress = revenue.merge(client_base, how = 'outer', on = 'client_id')
revenueaddress

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4
